<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
</center>

<h1> Aprendizaje Supervisado -> Grupo 5 Cohorte 2</h1>

Patricia Loto

Sandra Monica Olariaga

Veronica Bornancini

Fernandez María Soledad   


# Diplodatos Kaggle Competition

We present this peace of code to create the baseline for the competition, and as an example of how to deal with these kind of problems. The main goals are that you:

1. Learn
2. Try different models and see which one fits the best the given data
3. Get a higher score than the given one in the current baseline example
4. Try to get the highest score in the class :)

In [ ]:
# Importamos las librerías necesarias
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats 
import seaborn as sns 
import warnings

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from IPython.display import display_html
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from imblearn.ensemble import BalancedBaggingClassifier

In [ ]:
# Leemos el dataset original
original_df = pd.read_csv('../data/train.csv')

# 1. Learn

## 1.1 Verificamos columnas con sus datos

In [ ]:
# 1) Averiguamos el nombre de cada variable presente en el dataset
original_df.columns

In [ ]:
original_df.shape

In [ ]:
# 2) Inspeccionamos las primeras filas del dataset
original_df[:5]

**Observaciones:** Contamos con 453.411 registros y 7 variables en nuestra base de datos;
* **TripType:** Es nuestra variable objetivo. Es una identificación categórica (id) que representa el tipo de viaje de compras que realizó el clienteEsta variable no estará presente en el conjunto de test.
* **VisitNumber:** Número de identificación del comprador o visitante.
* **Weekday:** Día de la semana en que se hizo la compra.
* **Upc:** Codigo de identifiación del producto, hace referencia al código de barras.
* **ScanCount:** Cantidad de unidades de ese item que el cliente compró. El valor negativo, indica que el item se devolvió
* **DepartmentDescription:** Es una descripción de alto nivel del departamento al que pertenece el artículo.
* **FinelineNumber:** Hace referencia a una categoría más refinada para cada uno de los productos.

In [ ]:
# 3) Consultamos el tipo de dato de cada variable
original_df.dtypes

In [ ]:
# 4) Pedimos una descripción de los datos. Por defecto se muestra la información de las variables numéricas
original_df.describe().round()

In [ ]:
# 5) Como tenemos variables tipo Object, pedimos ver un resumen del resto de los datos (no numéricos)
original_df.describe(include=['object'])

**Observaciones:**
* La mayor compra de un item, fue de 71 unidades
* El día más cocurrido para hacer compras es el domingo
* Los productos más elegidos son los categorizados como **Grocery dry goods**
* Los valores de conteo total por variable no coinciden, lo que nos habla de que hay variables con datos nulos.

In [ ]:
# 6) Chequeamos que no haya caracteres fuera de a-Z, 0-9 y _ en los nombres de las variables/columnas
original_df.columns[~original_df.columns.str.match(r'^(\w+)$')]

In [ ]:
# 7) Averiguamos los posibles valores que toman las variables tipo Object
# Variable Weekday
set(original_df.Weekday)

In [ ]:
# Variable DepartmentDescription
set(original_df.DepartmentDescription)

In [ ]:
# 8) Cuantificamos por Tipo de Viaje (TripType)
s_triptype = original_df.groupby("VisitNumber").TripType.mean().value_counts()
df_triptype = s_triptype.to_frame(name="TripType")
df_triptype = df_triptype.sort_values('TripType')
df_triptype_styler = df_triptype.style.set_table_attributes("style='display:inline;padding-left:20px;'").set_caption('Cantidad por TRIPTYPE')
display_html(df_triptype_styler._repr_html_(), raw=True)

In [ ]:
# 9) Agrupamos por VisitNumber para realizar un conteo de los valores de la variable TripType 
plt.figure (figsize=(8,6))
plt.title('Cantidad de Tickets por TripType')
sns.despine(left=True)
original_df.groupby("VisitNumber").TripType.mean().value_counts().plot(kind='bar')

* **Observación:** Los datos se encuentran desbalanceados con respecto a la cantidad de registros por TripType

In [ ]:
# 10) Graficamos TripType agrupado por Weekday
plt.figure (figsize=(6,4))
plt.title('TripType agrupado por Weekday')
sns.countplot(data=original_df, x='Weekday', color='dodgerblue')
plt.xticks(rotation=90)
plt.ylabel('Quantity')
sns.despine(left=True)

## 1.2 Verificacion de Valores Nulos

In [ ]:
# 1) Obtenemos el número de valores faltantes por columna(variables) 
missing_values_count = original_df.isnull().sum()
missing_values_count[missing_values_count > 0]   

In [ ]:
# 2) Chequeamos si los registros con valores nulos de DepartmentDescription, se corresponden con los de Upc y FinelineNumber
(original_df.DepartmentDescription.isnull().sum(),
 (original_df.DepartmentDescription.isnull() & original_df.Upc.isnull() & original_df.FinelineNumber.isnull()).sum()) # si es nan el departamento los otros dos atributos lo son

In [ ]:
# 3) Chequeamos si los registros con valores nulos de Upc, se corresponden con los de FinelineNumber
(original_df.Upc.isnull().sum(),
 original_df.FinelineNumber.isnull().sum(),
 (original_df.FinelineNumber.isnull() & original_df.Upc.isnull()).sum())

In [ ]:
# 4) Eliminamos los registros que tengan valores nulos para las 3 variables
df_notna = original_df[pd.notnull(original_df['DepartmentDescription'])]
print(df_notna.DepartmentDescription.isnull().sum())
print(df_notna.FinelineNumber.isnull().sum())
print(df_notna.Upc.isnull().sum())

In [ ]:
print("Cantidad de Registros iniciales;", len(original_df))
print("Cantidad de registros removidos:", len(original_df) - len(df_notna))
print("Cantidad de Registros actuales:", len(df_notna))

In [ ]:
# 5) Reemplazamos los NaN de la feature UPC por la moda ya que se trata de una variable numerica discreta
warnings.filterwarnings('ignore')
df_notna.Upc.mode()[0]

df_notna.fillna({'Upc': df_notna.Upc.mode()[0]}, inplace=True)
df_notna.Upc.isnull().sum()

In [ ]:
# 5) Reemplazamos los NaN de la feature FinelineNumber por la moda ya que se trata de una variable numerica discreta
warnings.filterwarnings('ignore')
df_notna.FinelineNumber.mode()[0]

df_notna.fillna({'FinelineNumber': df_notna.FinelineNumber.mode()[0]}, inplace=True)
df_notna.FinelineNumber.isnull().sum()

**Observaciones:** 
* Tenemos tres variables con valores nulos (Upc, DepartmentDescription y FinelineNumber).
* La cantidad de nulos para las variables Upc y FinelineNumber es el mismo, y los registros se corresponden.
* Verificamos que cuando DepartmentDescription es NaN, las columnas Upc y FinelineNumber también lo son, por lo tanto removemos esos registros.
* Reemplazamos por la moda, los valores nulos de las variables Upc y FinelineNumber.

## 1.3 Verificacion y eliminacion de Outliers

In [ ]:
# 1) Graficamos los boxplot para detectar outliers
for feature in df_notna.columns:
    if feature != 'TripType':
        plt.figure (figsize=(15,8))
        sns.boxplot(data=df_notna, color='dodgerblue', x='TripType', y=feature)
        plt.xlim(0, 50)
        plt.ylabel(feature)
        plt.xlabel('TripType')
        plt.show()

**Observaciones:**
* Para la variable **VisitNumber** se detectan muy pocos outliers en los Triptype 12 y 14. Pero al ser una variable identificadora (id) no se considerarán como outliers esos valores.
* La variable **Upc** posee muhcos outliers para la mayoría de los diferentes Triptype. 
* La variable **FinelineNumber** que también es un código de clasificación, posee menos outliers que **Upc**.
* Podemos notar que para la variable **ScanCount** existen outliers para algunos Tipos de Viaje (TripType).
* La variable  **DepartmentDescription** posee muchas categorías con outliers.

In [ ]:
# 3) Filtramos los outliers 
def clean_outliers(dataset, column_name):
    """Returns dataset removing the outlier rows from column @column_name."""
    interesting_col = dataset[column_name]
    mask_var_outlier = (
        np.abs(interesting_col - interesting_col.mean()) <= (15 * interesting_col.std()))
    return dataset[mask_var_outlier]

In [ ]:
# Filtramos los outliers de la variable ScanCount
df_clean_ScanCount = clean_outliers(df_notna, "ScanCount")
print("Filas removidas", len(df_notna) - len(df_clean_ScanCount))

In [ ]:
# Redefiimos el nombre del dataset y calculamos el total de filas removidas hasta el momento
df_clean = df_clean_ScanCount
print("Cantidad de registros iniciales", len(original_df))
print("Total de registros filtrados", len(original_df) - len(df_clean))
print("Total de registros actuales", len(df_clean))

In [ ]:
# 4) Graficamos los boxplot para corroborar la limpieza de los outliers de la variable ScanCount

plt.figure (figsize=(17,6))

plt.subplot(121)
plt1=sns.boxplot(data=original_df, color='green', x='TripType', y='ScanCount')
plt.title('ScanCount Original')
plt.ylabel('ScanCount')
plt.xlabel('TripType')

plt.subplot(122)
plt2=sns.boxplot(data=df_clean, color='green', x='TripType', y='ScanCount')
plt.title('ScanCount Filtrado')
plt.ylabel('ScanCount')
plt.xlabel('TripType')

In [ ]:
# 5) Agrupamos los registros por VisitNumber
df_group = original_df.groupby(by='VisitNumber', as_index=False).agg({'Weekday': pd.Series.nunique})
print(df_group['Weekday'].unique())

In [ ]:
df_group = original_df.groupby(by='VisitNumber', as_index=False).agg({'TripType': pd.Series.nunique})
print(df_group['TripType'].unique()) 

**Observaciones:**
* Al agrupar por **VisitNumber** y verificar los valores de Weekday notamos que todos los tickets son únicos para la variable Weekday
* Indica que Cada Visitante realizó una única compra en un mismo día.
* Al agrupar por **VisitNumber** y verificar los valores de TripType notamos tambien que todos los tickets son únicos para esta feature

In [ ]:
#y = df_clean.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType
#y.shape

In [ ]:
#bins = set(df_clean.TripType)

## 1.4  Luego del análisis exploratorio, presentamos las consideraciones importantes que tomaremos en cuenta para la creación de la función que limpie y cargue el nuevo dataset:
* **1)** Usaremos **One hot encoding** para las variables **Weekday** y **DepartmentDescription**. Todas las transformaciones son aplicadas a los sets de Train y Test.
* **2)** No filtraremos los valores nulos de **DepartmentDescription**.
* **3)** Reemplazamos los valores nulos de la variable **Upc**  por la moda, ya que se trata de una variable numérica discreta.
* **4)** Es posible que tengamos varios registros para una sola visita (VisitNumber) y el objetivo es clasificar todos esos registros exactamente de la misma manera. Por tanto, prepararemos los datos de forma que toda la información de una visita quede en el mismo registro.
* **5)** Eliminaremos los outliers de las variables **ScanCount** con más de 15 desviaciones de la media
* **6)** Contaremos la variable **DepartmentDescription** para todos los artículos adquiridos en la misma visita.
* **7)** Para probar los distintos modelos tomaremos la mitad del dataset en forma aleatoria.

In [ ]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname) # Cargamos el dataset de Entrenamiento
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname) # Cargamos el dataset de Evaluación
    df_test['is_train_set'] = 0
   
      # En el set de Entrenamiento
    # Agrupamos el dataset por VisitNumber y obtenemos el máximo
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType

    # Removemos la variable TripType
    # Unificamos los conjuntos de datos (train y test)
    # Utilizamos la función concat para mantener el mismo índice de los registros
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # Reemplazamos los valores nulos de Upc y Finelinenumber
    df.fillna({'Upc': original_df.Upc.mode()[0]}, inplace=True)
    df.fillna({'FinelineNumber': 0}, inplace=True)

    # Aplicamos one-hot encoding para la variable DepartmentDescription sin valores nulos
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)

    # Agrupamos por visitNumber
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    # Aplicamos one-hot encoding par la variable Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)

    # Obtenemos los sets de Entrenamiento y Evaluación
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

In [ ]:
# Cargamos la información del dataset
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")

In [ ]:
X.shape, y.shape, XX.shape

### 2. Prueba de diferentes modelos para seleccionar cual ajusta mejor a nuestros datos

**DESBALANCEO DE DATOS**
Podemos optar por alguno de estos dos metodos para balancearlos

*Ajuste de Parámetros del modelo*: Consiste en ajustar parametros ó metricas del propio algoritmo para intentar equilibrar a la clase minoritaria penalizando a la clase mayoritaria durante el entrenamiento. Ejemplos on ajuste de peso en árboles, también en logisticregression tenemos el parámetro class_weight= “balanced” que utilizaremos en este ejemplo. No todos los algoritmos tienen estas posibilidades. En redes neuronales por ejemplo podríamos ajustar la métrica de Loss para que penalice a las clases mayoritarias.

*Balanced Ensemble Methods*: Utiliza las ventajas de hacer ensamble de métodos, es decir, entrenar diversos modelos y entre todos obtener el resultado final (por ejemplo “votando”) pero se asegura de tomar muestras de entrenamiento equilibradas.

# Creación y Evaluación del modelo

#### 2. Selección del Modelo: **Random Forest**

##### 2.1 Dividir los dataset en entrenamiento y test

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# El dataframe results será utilizado para almacenar los resultaados computados
results = pd.DataFrame(columns=('clf', 'best_acc'))

##### 2.2 Setear Hiperparametros, entrenar y seleccionar el Mejor Modelo

In [ ]:
# Seteo de Hiperparametros
rf_param = {
              'criterion':('gini', 'entropy'), 
              'min_samples_leaf':(1, 5, 10),
              'min_samples_split':(2, 3, 5, 10, 20, 30),
              'random_state': [0,1,2],
              'n_estimators': [50,75,100],
              'class_weight' : ('balanced', 'balanced_subsample'),
              'random_state': [0,1,2]
            }

model_rf = RandomForestClassifier()
rf_clf = GridSearchCV(model_rf, rf_param, cv=3, scoring='accuracy') #scoring='balanced_accuracy')

In [ ]:
# Entrenamiento del modelo
rf_clf.fit(X_train, y_train)

In [ ]:
# Seteamos un nuevo dataframe llamado results_rf, para almacenar los resultados computados
results_rf = rf_clf.cv_results_
    
df = pd.DataFrame(results_rf
df_result = df[['param_criterion', 'param_min_samples_leaf', 'param_min_samples_split', 'param_random_state', 'param_n_estimators', 'param_class_weight', 'mean_test_score', 'std_test_score', 'rank_test_score']]
df_result

In [ ]:
# Seleccionamos el modelo con mean_test_score mas alto y menor varianza
df_bm = df_result[df_result['rank_test_score'] == 1]
df_bm

In [ ]:
best_rf_clf = rf_clf.best_estimator_
bestpar = rf_clf.best_params_

print('Mejor Modelo\n', best_rf_clf)
print('Mejores Parametros\n', bestpar)

In [ ]:
print('Best Random Forest accuracy: ', rf_clf.best_score_)
results = results.append({'clf': best_rf_clf, 'best_acc': rf_clf.best_score_}, ignore_index=True)

In [ ]:
results

In [ ]:
print(results.loc[results['best_acc'].idxmax()]['clf'])

In [ ]:
# Realizamos predicciones para el set de Evaluación
X.shape, XX.shape

In [ ]:
yy = results.clf.iloc[0].predict(XX)

In [ ]:
yy

In [ ]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [ ]:
submission.to_csv("../data/submission_4.csv", header=True, index=False)

In [ ]:
submission